## 장르 기반 유사도로 추천

In [2]:
import pandas as pd
import numpy as np


In [3]:
movies = pd.read_csv('./data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(2)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [4]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [5]:
# 장르값이 여러개가 들어있어서 literal_eval로 tuple로 변환
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

C:\Users\oem\AppData\Local\Temp\ipykernel_7480\3118422419.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(literal_eval)
C:\Users\oem\AppData\Local\Temp\ipykernel_7480\3118422419.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


In [6]:
# genres컬럼에서 장르명만, keywords컬럼에서 키워드만 리스트로 넣기
# 리스트 내 여러개의 딕셔너리의 'name'키에 해당하는 값을 찾아서 리스트 형태로 변환
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df.head(2)

C:\Users\oem\AppData\Local\Temp\ipykernel_7480\2997131239.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
C:\Users\oem\AppData\Local\Temp\ipykernel_7480\2997131239.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])


,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha..."


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# 장르값들을 벡터화 하기위해 띄어쓰기로 구분되는 문장으로 변환.
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df.head(2)

C:\Users\oem\AppData\Local\Temp\ipykernel_7480\1510327378.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action


In [8]:
# min_df로 지정한 값보다 작은 경우에는 무시 
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2)) # ngram 단어의 묶음을 1개부터 2개까지 설정(science Fiction같은 2개 묶음)
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
genre_mat.shape
#4803,276 피처벡터행렬

(4803, 276)

In [9]:
# 생성한 행렬에 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:5])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 [0.12598816 0.16903085 0.3380617  ... 0.12598816 0.         0.        ]
 [0.75592895 0.3380617  0.50709255 ... 0.         0.         0.        ]]


In [10]:
# 유사도가 가장 높은 순으로 정렬
genre_sim_sorted = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted[:1])
# 0번 레코드는 3494번 레코드가 유사도 가장 높고, 가장 낮은 레코드는 2401번이다.

[[   0 3494  813 ... 3038 3037 2401]]


In [11]:
def find_movie(df, sorted_index, title_name, top_list=10): # top_list : 상위 10개만 추천

    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    # genre_sim_sorted에서 유사도 순으로 top_list개 index추출
    similar_index = sorted_index[title_index, :(top_list)]
    
    # top_list index출력. (2차원->1차원으로)
    print(similar_index)
    similar_index = similar_index.reshape(-1)
    
    return df.iloc[similar_index]

In [12]:
movies_df[movies_df['title'] == 'The Godfather']

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
3337,238,The Godfather,"[Drama, Crime]",8.4,5893,143.659698,"[italy, love at first sight, loss of father, p...","Spanning the years 1945 to 1955, a chronicle o...",Drama Crime


In [13]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]
# 평균평점순으로 정렬했더니 투표수를 보니 평가횟수가 1회

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [14]:
similar_movies = find_movie(movies_df, genre_sim_sorted, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]
# kids는 장르가 아예 다름, 평점이 0인 영화도 있음

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [15]:
# 가중평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
# v: 개별 영화에 평점을 투효한 횟수
# m: 평점을 부여하기 위한 최소 투표 횟수
# R: 개별 영화에 대한 평균 평점
# C: 전체 영화에 대한 평균 평점

# v = 'vote_count'
# R = 'vote_average'
# C = movies_df['vote_average'].mean()

# m = 값이 높으면 평점 투표 횟수가 많은 영화에 높은 가중평점을 부여. 
# 상위 60%에 해당하는 횟수를 기준

C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C,3), 'm:', round(m,3))

C: 6.092 m: 370.2


In [16]:
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m)) * R) + ((m/(v+m)) * C)

movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

C:\Users\oem\AppData\Local\Temp\ipykernel_7480\2815177072.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)


In [17]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [18]:
def find_sim_movie(df, sorted_idx, title_name, top_list=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # 장르 유사성이 높은 영화 top_list의 2배수만큼 후보선정
    similar_index = sorted_idx[title_index, :(top_list*2)]
    similar_index = similar_index.reshape(-1)
    
    similar_index = similar_index[similar_index != title_index] # 기존 영화 인덱스 제외
    
    return df.iloc[similar_index].sort_values('weighted_vote', ascending=False)[:top_list]



In [36]:
similar_movie = find_sim_movie(movies_df,genre_sim_sorted,'The Godfather',10)
similar_movie[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


## Surprise (협업시스템 기반)

- Surprise 는 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트만 적용할 수 있다. 
- 그래서 데이터의 첫번째 컬럼을 사용자 아이디, 두번째 컬럼을 아이템 아이디, 세번째 컬럼을 평점으로 가정해 데이터를 로딩하고 네번째 컬럼부터는 로딩을 수행하지 않는다. 
    - 예로, user_id, item_id, rating, time_stamp 필드로 구분된 데이터라면 앞 3개 필드만 로딩하고 이후 time_stamp 필드는 로딩에서 제외
    - 따라서 반드시 데이터 세트의 컬럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 돼 있어야 한다.

https://big-dream-world.tistory.com/70?category=470634

### 1. Surprise를 이용한 추천 시스템 구축

- http://sruprise.readthedocs.io/en/stable/ 에 정리되어있다
- 간단한 예제 : 추천데이터를 학습용과 테스트용 데이터세트로 분리한 뒤 SVD 행렬분해를 통해 잠재요인협업필터링 수행하기

In [37]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [38]:
data = Dataset.load_builtin('ml-100k')
# 수행시마다 동일하게 데이터를 분할하기 위해 random_state값 부여
trainset, testset = train_test_split(data, test_size=0.2, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\oem/.surprise_data/ml-100k


- surprise는 자체적으로 로우레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 데이터를 데이터 세트로 적용해야한다.

In [39]:
# SVD로 잠재 요인 협업 필터링 수행
algo=SVD() # 알고리즘 객체 생성
algo.fit(trainset)

- surprise에서 추천을 예측하는 메서드는 test()와 predict()이다.
- test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드
- predict()는 개별 사용자와 영화에 대한 추천 평점을 반환하는 메서드

In [40]:
# 테스트데이터 세트에 대해 추천 수행
predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:',len(predictions))
print('prediction 결과의 최초 5개 추출:')
predictions[:5]
# 개별 사용자 아이디(uid), 영화(아이템)아이디(iid), 실제 평점(r_ui)정보에 기반해 surprise의 추천 예측 평점(est)데이터를 튜플 형태로 갖고있다.
# was_impossible이 True면 예측값을 생성할 수 없는 데이터라는 의미이다.

prediction type: <class 'list'> size: 20000
prediction 결과의 최초 5개 추출:


[Prediction(uid='269', iid='17', r_ui=2.0, est=3.055237625244456, details={'was_impossible': False}),
 Prediction(uid='704', iid='382', r_ui=4.0, est=3.6424451121227026, details={'was_impossible': False}),
 Prediction(uid='829', iid='475', r_ui=4.0, est=4.070547110893363, details={'was_impossible': False}),
 Prediction(uid='747', iid='274', r_ui=4.0, est=3.253029720506131, details={'was_impossible': False}),
 Prediction(uid='767', iid='98', r_ui=5.0, est=4.892030535597849, details={'was_impossible': False})]

In [41]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('269', '17', 3.055237625244456),
 ('704', '382', 3.6424451121227026),
 ('829', '475', 4.070547110893363)]

In [43]:
# predict로 추천예측해보기
# 사용자 아이디, 아이템 아이디를 문자열로 입력 -> 추천 예측 평점을 est로 반환
uid=str(196)
iid=str(302)
pred=algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.05   {'was_impossible': False}


In [44]:
accuracy.rmse(predictions)

RMSE: 0.9475


0.9474769857562015

### 02. OS파일 데이터를 surprise데이터 세트로 로딩

In [64]:
import pandas as pd
ratings=pd.read_csv('./data/ratings_small.csv')
# ratings_noh.csv파일로 언로드시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('./data/ratings_noh.csv', index=False, header=False)

- Reader 클래스의 주요 생성 파라미터
    - line_format(string) : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식
    - sep(char) : 칼럼을 분리하는 분리자(문자), 디폴트는 '\t'
    - rating_scale(tuple,optional) : 평점 값의 최소~ 최대 설정. 디폴트는(1,5)

In [65]:
from surprise import Reader

# 컬럼의 분리문자는 콤마, 평점 단위는 0.5, 최대평점 5
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./data/ratings_noh.csv', reader=reader)

In [66]:
# SVD 행렬분해기법을 이용해 추천 예측 수행
trainset, testset = train_test_split(data, test_size=0.2, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정
# 잠재요인 크기 K값을 나타내는 파라미터인 n_facctors 50으로 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 테스트 데이터로 평점 예측 후 rmse 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8851


0.8850646057753477

### 03. DataFrame에서 surprsie 데이터 세트로 로딩
- 주의할 점은 사용자 아이디, 아이템 아이디, 평점 컬럼 순서를 지켜야 한다.

In [67]:
import pandas as pd
from surprise import Reader, Dataset
 
ratings = pd.read_csv('./data/ratings_small.csv')
reader = Reader(rating_scale=(0.5, 5.0))
 
# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=0)


In [68]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8851


0.8850646057753477

### 04. 교차검증과 하이퍼 파라미터 튜닝

- Surprise 는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate() 와 GridSearchCV 클래스를 제공한다.
- cross_validate()는 인자로 알고리즘 객체, 데이터, 성능 평가 방법, 폴드 데이터 세트 개수(cv)를 입력받아, 출력결과와 같이 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여준다.

In [69]:
from surprise.model_selection import cross_validate
 
# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩
ratings = pd.read_csv('./data/ratings_small.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
 
algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# 폴드별 성능평가수치와 전체 폴드의 평균 성능평가수치를 출력

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8880  0.8962  0.8938  0.9060  0.9002  0.8968  0.0061  
MAE (testset)     0.6860  0.6892  0.6879  0.6964  0.6929  0.6905  0.0037  
Fit time          1.03    1.04    1.04    1.05    1.05    1.04    0.01    
Test time         0.17    0.17    0.17    0.17    0.17    0.17    0.00    


{'test_rmse': array([0.88799167, 0.89617246, 0.89384963, 0.90601185, 0.90019382]),
 'test_mae': array([0.68603608, 0.68924295, 0.68789896, 0.69635425, 0.69288619]),
 'fit_time': (1.0288190841674805,
  1.0438623428344727,
  1.0370919704437256,
  1.0455753803253174,
  1.0459282398223877),
 'test_time': (0.166062593460083,
  0.16726326942443848,
  0.16634368896484375,
  0.166672945022583,
  0.17114830017089844)}

- Surprise 는 GridSearchCV를 통해 교차 검증을 통한 하이퍼 파라미터 최적화를 수행할 수 있다. 
- 하이퍼 파라미터 최적화는 알고리즘 유형에 따라 다를 수 있지만, SVD의 경우 주로 SGD(점진적 하강 방식)의 반복 횟수를 지정하는 n_epochs와 SVD의 잠재 요인 K의 크기를 지정하는 n_factors를 튜닝한다. 

In [70]:
from surprise.model_selection import GridSearchCV
 
# 최적화할 파라미터들을 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }
 
# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
 
# CV가 3일때 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
# n_epochs:20, n_factors:50일때 3개 폴드의 검증 데이터 세트에서 최적 RMSE가 약 0.8994로 도출되었다.

0.8994010782422769
{'n_epochs': 20, 'n_factors': 50}


In [71]:
from surprise.dataset import DatasetAutoFolds
 
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='./data/ratings_noh.csv', reader=reader)
 
#전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()
 
 
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)
 
 
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./data/movies.csv')
 
# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
 
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
40       42  Dead Presidents (1995)  Action|Crime|Drama


In [72]:
uid = str(9)
iid = str(42)
 
pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 2.64   {'was_impossible': False}


In [73]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
     
    # 모든 영화들의 movieId를 리스트로 생성.
    total_movies = movies['movieId'].tolist()
     
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
     
    return unseen_movies
 
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 45 추천대상 영화수: 9080 전체 영화수: 9125


In [74]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
     
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
     
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
     
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
     
    return top_movie_preds
 
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')
 
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 45 추천대상 영화수: 9080 전체 영화수: 9125
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.400579409001147
Godfather, The (1972) : 4.370540045092638
It Happened One Night (1934) : 4.363849637541674
Casablanca (1942) : 4.336024874543918
Maltese Falcon, The (1941) : 4.321015160843752
All About Eve (1950) : 4.309431147405599
It's a Wonderful Life (1946) : 4.303661756312046
African Queen, The (1951) : 4.289780175135358
Cinema Paradiso (Nuovo cinema Paradiso) (1989) : 4.283687861376393
Modern Times (1936) : 4.271531364943996
